## Настройка окружения
Установим все необходимые зависимости

In [3]:
!pip install beautifulsoup4 requests readability-lxml python-telegram-bot httpx python-dotenv
!pip install nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.1/717.1 kB 19.2 MB/s eta 0:00:00


In [4]:
import nest_asyncio
nest_asyncio.apply()

In [13]:
from google.colab import userdata
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
TELEGRAM_BOT_TOKEN = userdata.get('TELEGRAM_BOT_TOKEN')

## 1. Парсер статей

In [5]:
from bs4 import BeautifulSoup
import requests
from readability import Document
import re

def extract_article_text(url: str) -> str:
    """
    Извлекает чистый текст статьи из URL
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }

        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        doc = Document(response.text)
        soup = BeautifulSoup(doc.summary(), 'html.parser')

        # Очищаем текст от лишних пробелов
        clean_text = soup.get_text()
        clean_text = re.sub(r'\s+', ' ', clean_text).strip()

        return clean_text[:5000]  # Ограничиваем длину для прототипа

    except Exception as e:
        print(f"Ошибка парсинга: {e}")
        return None


In [6]:
# Тестируем парсер
test_url = "https://habr.com/ru/articles/789322/"
text = extract_article_text(test_url)
print(f"Извлечено символов: {len(text) if text else 0}")
if text:
    print(f"Превью: {text[:200]}...")

Извлечено символов: 5000
Превью: Общий вид САПР Delta Design для .NET 6САПР Delta Design — это отечественная ECAD система для проектирования электронных устройств от компании ЭРЕМЕКС, имеющая десятилетнюю историю. Версия Delta Design...


## 2. LLM-интеграция (DeepSeek API)

In [7]:
import httpx
import json
import asyncio

class DeepSeekAnalyzer:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://api.deepseek.com/v1/chat/completions"

    async def analyze_article(self, text: str) -> dict:
        """
        Анализирует текст статьи с помощью DeepSeek API
        """
        prompt = """Проанализируй техническую статью и верни JSON со следующими полями:
        - tags: список из 3-5 тегов (например, ["Python", "API", "Machine Learning"])
        - summary: краткий пересказ на русском (50-100 слов)
        - thesis: основной тезис статьи (1-2 предложения)
        - category: категория (Programming, DevOps, Data Science, AI, Web Development, Other)

        Текст статьи: {text}

        Ответ только в формате JSON, без дополнительного текста.""".replace("{text}", text[:3000])

        try:
            async with httpx.AsyncClient(timeout=30.0) as client:
                response = await client.post(
                    self.base_url,
                    headers={
                        "Authorization": f"Bearer {self.api_key}",
                        "Content-Type": "application/json"
                    },
                    json={
                        "model": "deepseek-chat",
                        "messages": [
                            {
                                "role": "system",
                                "content": "Ты эксперт по анализу технических текстов. Отвечай только валидным JSON."
                            },
                            {
                                "role": "user",
                                "content": prompt
                            }
                        ],
                        "temperature": 0.3,
                        "max_tokens": 1000
                    }
                )

                if response.status_code == 200:
                    result = response.json()
                    content = result["choices"][0]["message"]["content"]

                    # Пытаемся извлечь JSON из ответа
                    json_match = re.search(r'\{[\s\S]*\}', content)
                    if json_match:
                        return json.loads(json_match.group())
                    return {"error": "Не удалось распарсить JSON ответ"}

                else:
                    return {"error": f"API error: {response.status_code}"}

        except Exception as e:
            return {"error": f"Exception: {str(e)}"}


## 3. Основной обработчик

In [14]:
class ArticleProcessor:
    def __init__(self, api_key: str):
        self.analyzer = DeepSeekAnalyzer(api_key)

    async def process_url(self, url: str) -> dict:
        """
        Основной метод обработки URL
        """
        # Извлекаем текст
        text = extract_article_text(url)
        if not text:
            return {"error": "Не удалось извлечь текст статьи"}

        # Анализируем через LLM
        analysis = await self.analyzer.analyze_article(text)

        if "error" not in analysis:
            # Формируем результат
            result = {
                "url": url,
                "text_preview": text[:500] + "..." if len(text) > 500 else text,
                "analysis": analysis
            }
            return result
        else:
            return {"error": analysis["error"]}


## 4. Тестирование системы

In [23]:
# Инициализируем процессор
processor = ArticleProcessor(DEEPSEEK_API_KEY)

# %%
# Тестовые URL для проверки
test_urls = [
    "https://habr.com/ru/articles/789322/",  # Про программирование
    "https://habr.com/ru/articles/789150/",  # Про DevOps
]

# %%
# Запускаем тестовую обработку
async def test_processing():
    for url in test_urls:
        print(f"\n🔗 Обрабатываем: {url}")
        result = await processor.process_url(url)

        if "error" in result:
            print(f"❌ Ошибка: {result['error']}")
        else:
            print("✅ Успешно обработано!")
            print(f"📊 Результат:")
            print(f"   Теги: {', '.join(result['analysis'].get('tags', []))}")
            print(f"   Категория: {result['analysis'].get('category', 'Unknown')}")
            print(f"   Пересказ: {result['analysis'].get('summary', '')[:100]}...")

# Запускаем тест
await test_processing()


🔗 Обрабатываем: https://habr.com/ru/articles/789322/
❌ Ошибка: API error: 402

🔗 Обрабатываем: https://habr.com/ru/articles/789150/
❌ Ошибка: API error: 402


## 5. Telegram-бот

In [19]:
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class TelegramBot:
    def __init__(self, token: str, processor: ArticleProcessor):
        self.token = token
        self.processor = processor
        self.application = Application.builder().token(token).build()

        # Регистрируем обработчики
        self.application.add_handler(CommandHandler("start", self.start_handler))
        self.application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, self.message_handler))

    async def start_handler(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Обработчик команды /start"""
        welcome_text = """
        🤖 Добро пожаловать в ArtiFact Prototype!

        Отправьте мне ссылку на техническую статью (Habr, Medium, arXiv и др.), и я:
        - Извлеку ключевые теги
        - Сделаю краткий пересказ
        - Определю категорию

        Просто отправьте URL! 🚀
        """
        await update.message.reply_text(welcome_text)

    async def message_handler(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Обработчик текстовых сообщений"""
        text = update.message.text

        # Проверяем, является ли текст URL
        if not text.startswith(('http://', 'https://')):
            await update.message.reply_text("📎 Пожалуйста, отправьте валидный URL (начинается с http:// или https://)")
            return

        # Отправляем статус обработки
        status_msg = await update.message.reply_text("⏳ Обрабатываю статью...")

        try:
            # Обрабатываем статью
            result = await self.processor.process_url(text)

            if "error" in result:
                await status_msg.edit_text(f"❌ Ошибка: {result['error']}")
                return

            analysis = result['analysis']

            # Форматируем ответ
            response = f"""
            📊 **Результат анализа**

            🔖 **Теги:** {', '.join(analysis.get('tags', []))}
            📁 **Категория:** {analysis.get('category', 'Unknown')}

            📝 **Краткий пересказ:**
            {analysis.get('summary', '')}

            💡 **Основной тезис:**
            {analysis.get('thesis', '')}
            """

            await status_msg.edit_text(response)

        except Exception as e:
            await status_msg.edit_text(f"❌ Произошла ошибка: {str(e)}")

    def run(self):
        """Запускает бота"""
        logger.info("Запускаем бота...")
        self.application.run_polling()



## 6. Альтернативные обработчики

### 6.1 Локальная LLM через Ollama

In [20]:
class OllamaAnalyzer:
    async def analyze_article(self, text: str) -> dict:
        """
        Альтернатива: использование локальной LLM через Ollama
        """
        try:
            async with httpx.AsyncClient(timeout=60.0) as client:
                response = await client.post(
                    "http://localhost:11434/api/generate",
                    json={
                        "model": "mistral",  # или "llama3", "mixtral"
                        "prompt": f"Проанализируй техническую статью и верни JSON с полями: tags (список тегов), summary (краткий пересказ), thesis (основной тезис). Текст: {text[:2000]}",
                        "format": "json",
                        "stream": False
                    }
                )

                if response.status_code == 200:
                    return response.json().get("response", {})
                else:
                    return {"error": f"Ollama error: {response.status_code}"}

        except Exception as e:
            return {"error": f"Ollama exception: {str(e)}"}

### Вариант 6.2: Hugging Face Inference API

In [21]:
from transformers import pipeline

class HuggingFaceAnalyzer:
    def __init__(self):
        self.api_url = "https://api-inference.huggingface.co/models"

    async def analyze_article(self, text: str) -> dict:
        """
        Анализирует текст статьи с помощью бесплатного HF API
        """
        # Используем модель для суммаризации
        try:
            async with httpx.AsyncClient(timeout=60.0) as client:
                # Сначала получаем суммаризацию
                summary_response = await client.post(
                    f"{self.api_url}/facebook/bart-large-cnn",
                    headers={"Authorization": "Bearer hf_your_token_here"},
                    json={"inputs": text[:1024]}
                )

                if summary_response.status_code == 200:
                    summary = summary_response.json()[0]['summary_text']

                    # Простой анализ тегов (на основе ключевых слов)
                    tags = self.extract_tags(text)

                    return {
                        "tags": tags,
                        "summary": summary,
                        "thesis": summary[:150] + "...",  # Укороченная версия
                        "category": self.detect_category(text)
                    }
                else:
                    return {"error": f"HF API error: {summary_response.status_code}"}

        except Exception as e:
            return {"error": f"Exception: {str(e)}"}

    def extract_tags(self, text: str) -> list:
        """Простое извлечение тегов по ключевым словам"""
        keywords = {
            'python': ['python', 'django', 'flask'],
            'javascript': ['javascript', 'node', 'react', 'vue'],
            'devops': ['docker', 'kubernetes', 'ci/cd', 'devops'],
            'ai': ['ai', 'machine learning', 'ml', 'neural network'],
            'web': ['web', 'http', 'api', 'rest'],
            'data': ['data', 'database', 'sql', 'nosql']
        }

        found_tags = []
        text_lower = text.lower()

        for tag, words in keywords.items():
            if any(word in text_lower for word in words):
                found_tags.append(tag)

        return found_tags[:3] or ["technology"]

    def detect_category(self, text: str) -> str:
        """Определение категории"""
        text_lower = text.lower()

        if any(word in text_lower for word in ['python', 'java', 'c++', 'programming']):
            return "Programming"
        elif any(word in text_lower for word in ['docker', 'kubernetes', 'devops', 'deploy']):
            return "DevOps"
        elif any(word in text_lower for word in ['ai', 'machine learning', 'neural', 'deep learning']):
            return "AI"
        elif any(word in text_lower for word in ['web', 'http', 'browser', 'frontend']):
            return "Web Development"
        else:
            return "Technology"


In [22]:
class AltArticleProcessor:
    def __init__(self):
        # Пробуем разные анализаторы по порядку
        self.analyzers = [
            # LocalAnalyzer(),  # Первый приоритет - локальный
            HuggingFaceAnalyzer(),  # Второй - HF API
            # OpenRouterAnalyzer()  # Третий - OpenRouter
        ]

    async def process_url(self, url: str) -> dict:
        """
        Основной метод обработки URL
        """
        # Извлекаем текст
        text = extract_article_text(url)
        if not text:
            return {"error": "Не удалось извлечь текст статьи"}

        # Пробуем разные анализаторы пока один не сработает
        for analyzer in self.analyzers:
            try:
                analysis = await analyzer.analyze_article(text)
                if "error" not in analysis:
                    # Формируем результат
                    result = {
                        "url": url,
                        "text_preview": text[:500] + "..." if len(text) > 500 else text,
                        "analysis": analysis
                    }
                    return result
            except Exception as e:
                print(f"Анализатор {type(analyzer).__name__} failed: {e}")
                continue

        return {"error": "Все анализаторы не сработали"}

In [24]:
# Инициализируем процессор
processor = AltArticleProcessor()

# %%
# Тестовые URL для проверки
test_urls = [
    "https://habr.com/ru/articles/789322/",  # Про программирование
    "https://habr.com/ru/articles/789150/",  # Про DevOps
]

# %%
# Запускаем тестовую обработку
async def test_processing():
    for url in test_urls:
        print(f"\n🔗 Обрабатываем: {url}")
        result = await processor.process_url(url)

        if "error" in result:
            print(f"❌ Ошибка: {result['error']}")
        else:
            print("✅ Успешно обработано!")
            print(f"📊 Результат:")
            print(f"   Теги: {', '.join(result['analysis'].get('tags', []))}")
            print(f"   Категория: {result['analysis'].get('category', 'Unknown')}")
            print(f"   Пересказ: {result['analysis'].get('summary', '')[:100]}...")

# Запускаем тест
await test_processing()


🔗 Обрабатываем: https://habr.com/ru/articles/789322/
❌ Ошибка: Все анализаторы не сработали

🔗 Обрабатываем: https://habr.com/ru/articles/789150/
❌ Ошибка: Все анализаторы не сработали


## 8. Запуск бота

In [ ]:
# # Инициализация и запуск
processor = ArticleProcessor(DEEPSEEK_API_KEY)
bot = TelegramBot(TELEGRAM_BOT_TOKEN, processor)

# # Запускаем бота
print("Запускаем Telegram-бота...")
bot.run()